In [ ]:
import pandas as pd
import sqlite3

In [ ]:
!mkdir -p data/raw

In [ ]:
# load everything from sql
# data/db.sqlite3 you can get from the latest kilterboard apk
conn = sqlite3.connect("data/db.sqlite3")
climbs = pd.read_sql_query("SELECT * FROM climbs", conn)
grades = pd.read_sql_query("SELECT * FROM difficulty_grades", conn)
stats = pd.read_sql_query("SELECT * FROM climb_stats", conn)
holds = pd.read_sql_query("SELECT * FROM holes", conn)
placements = pd.read_sql_query("SELECT * FROM placements", conn)
holds = pd.merge(placements, holds, left_on="hole_id", right_on="id")
holds.set_index("id_x", inplace=True)

In [ ]:
# merge and rename
df = pd.merge(climbs.drop("angle", axis=1), stats, left_on="uuid", right_on="climb_uuid")
df["average_grade"] = df["difficulty_average"].apply(lambda x: grades.loc[int(x) + 1, "boulder_name"])
df["font_grade"] = df["average_grade"].apply(lambda x: x.split("/")[0])
df["v_grade"] = df["average_grade"].apply(lambda x: x.split("/")[1])

In [ ]:
LAYOUT_ID = 1
MIN_ASCENTS = 2
MIN_QUALITY = 2

print(df.shape)
df = df[df["frames_count"] == 1]
print(df.shape)
df = df[df["is_listed"] == 1]
print(df.shape)
df = df[df["layout_id"] == 1]
print(df.shape)
df = df[df["quality_average"] >= MIN_QUALITY]
print(df.shape)
df = df[df["ascensionist_count"] >= MIN_ASCENTS].reset_index()
print(df.shape)

holds = holds[holds["layout_id"] == 1]  # only original boards
holds = holds[holds.index.to_series() < 3000]

In [ ]:
import math
from collections import Counter


class KilterPolice:
    """Punishes bad climbs."""

    def __init__(
        self,
        allowed_holds: set,
        allowed_colors: set = set([12, 13, 14, 15]),  # 12 is start, 14 is finish, 13 is blue, 15 is orange
        n_start_holds: tuple[int, int] = (1, 2),
        n_finish_holds: tuple[int, int] = (1, 2),
        n_total_holds: tuple[int, int] = (2, math.inf),
    ):
        self.allowed_holds = allowed_holds
        self.allowed_colors = allowed_colors
        self.n_start_holds = n_start_holds
        self.n_finish_holds = n_finish_holds
        self.n_total_holds = n_total_holds

    def check(self, frames: str) -> bool:
        """Check if the climb is valid."""
        colors = []
        for frame in frames.split("p")[1:]:  # split by holds
            hold, color = frame.split("r")  # split into hold id and color
            if int(hold) not in self.allowed_holds:
                return False
            if int(color) not in self.allowed_colors:
                return False
            colors.append(int(color))
        if len(colors) < self.n_total_holds[0] or len(colors) > self.n_total_holds[1]:
            return False
        counter = Counter(colors)
        if counter[12] < self.n_start_holds[0] or counter[12] > self.n_start_holds[1]:
            return False
        if counter[14] < self.n_finish_holds[0] or counter[14] > self.n_finish_holds[1]:
            return False
        return True

In [ ]:
kp = KilterPolice(set(holds.index))
df["valid"] = df["frames"].apply(kp.check)
df = df[df["valid"]]
df["angle"] = df["angle"]

In [ ]:
df.to_csv("data/raw/climbs.csv")
holds.to_csv("data/raw/holds.csv")
grades.to_csv("data/raw/grades.csv")